# ML 
Stephen Williams

### Import Libraries and Data

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import randint, uniform
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV
# from sklearn.svm import SVC
# from xgboost import XGBClassifier

In [3]:
df = pd.read_csv(r'C:\Users\sleep\OneDrive\Desktop\features.csv')

In [4]:
print(len(df))
df.head()

6288


,Unnamed: 0,id,clause,class,clause_clean,clause_tv,clause_tv_stemmed,aa,aba,abandon,...,pc92,pc93,pc94,pc95,pc96,pc97,pc98,pc99,pc100,tv_len
0,0,1588,18. Governing Law: This Agreement shall be gov...,0,governing law this agreement shall be govern...,"['governing', 'law', 'agreement', 'shall', 'ac...","['govern', 'law', 'agreement', 'shall', 'accor...",0.0,0.0,0.0,...,-0.045778,0.006222,-0.011611,0.032893,0.013421,-0.013765,0.006258,-0.020431,0.007072,46
1,1,1146,"1.8 Modification. We may modify, update, or di...",1,modification we may modify update or disco...,"['modification', 'may', 'modify', 'update', 'd...","['modif', 'may', 'modifi', 'updat', 'discontin...",0.0,0.0,0.0,...,0.003726,0.005088,-0.014056,-0.030365,-0.001253,-0.050148,0.029921,-0.008823,-0.031375,32
2,2,4792,Except as otherwise expressly provided in this...,0,except as otherwise expressly provided in this...,"['except', 'otherwise', 'expressly', 'provided...","['except', 'otherwis', 'expressli', 'provid', ...",0.0,0.0,0.0,...,0.076674,-0.073145,-0.018112,-0.085275,0.000391,0.017757,0.031794,-0.014902,-0.034318,43
3,3,2759,8.3. The benefit and burdens of this Ag...,1,the benefit and burdens of this agreement ...,"['benefit', 'agreement', 'may', 'assigned', 't...","['benefit', 'agreement', 'may', 'assign', 'tra...",0.0,0.0,0.0,...,-0.016398,0.020122,-0.007510,0.017530,-0.004489,-0.020484,0.020942,-0.010176,-0.007482,10
4,5,3926,"COMPANY warrants that the SOFTWARE will, for a...",1,company warrants that the software will for a ...,"['company', 'period', 'sixty', 'days', 'date',...","['compani', 'period', 'sixti', 'day', 'date', ...",0.0,0.0,0.0,...,-0.023020,0.017967,0.007866,-0.002551,0.017762,-0.009227,-0.004196,-0.025751,-0.000235,33


In [5]:
pca = df.iloc[:, 3272 - 101:-1]

In [24]:
y = df['class']
y.value_counts()

0    4838
1    1450
Name: class, dtype: int64

### Logistic Regression
Note: for whatever reason, it did not permit the use of 'neg_brier_score' as an evaluation metric

In [25]:
lr = LogisticRegression(solver = 'saga', random_state = 123)
C = uniform(loc=0, scale=10)
hyperparams = dict(C = C, penalty = ['l1', 'l2'])

In [26]:
mod1 = RandomizedSearchCV(lr, hyperparams, random_state = 123, n_iter = 460, scoring = 'f1', cv = 5)

In [27]:
mod1_best = mod1.fit(pca, y)

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

In [28]:
print('Best params:', mod1_best.best_estimator_.get_params())

Best params: {'C': 9.630044659922582, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l1', 'random_state': 123, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [29]:
print('Best f1:', mod1_best.best_score_)

Best f1: 0.3665789223362318


In [33]:
mod1_best = LogisticRegression(penalty = 'l1', C = 9.630044659922582, solver = 'saga', random_state = 123, )
lr_mod = mod1_best.fit(pca, y)
lr_predict = lr_mod.predict(pca)

array([1, 0, 0, ..., 0, 0, 1], dtype=int64)

In [47]:
print(classification_report(y, lr_predict))

              precision    recall  f1-score   support

           0       0.82      0.95      0.88      4838
           1       0.61      0.29      0.39      1450

    accuracy                           0.79      6288
   macro avg       0.71      0.62      0.63      6288
weighted avg       0.77      0.79      0.76      6288



### Random Forest

In [56]:
rf = RandomForestClassifier(criterion = 'gini', random_state = 123)
rf_params = {'max_depth': [1, 3, 5, 10],
             'n_estimators': [100, 200, 300],
             'min_samples_leaf': [1, 3, 5, 10, 15]}

In [57]:
mod2 = RandomizedSearchCV(rf, rf_params, random_state = 123, n_iter = 460, scoring = 'f1', cv = 5)

In [58]:
mod2_best = mod2.fit(pca, y)

c:\users\sleep\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 60 is smaller than n_iter=460. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

In [59]:
print('Best params:', mod2_best.best_estimator_.get_params())

Best params: {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False}


In [60]:
print('Best f1:', mod2_best.best_score_)

Best f1: 0.14442459466544338


In [61]:
mod2_best = RandomForestClassifier(
                                   criterion = 'gini',
                                   max_depth = 10,
                                   max_leaf_nodes = None,
                                   min_samples_leaf = 1,
                                   min_samples_split = 2,
                                   n_estimators = 100,
                                   random_state = 123)
rf_mod = mod2_best.fit(pca, y)
rf_predict = rf_mod.predict(pca)

In [62]:
print(classification_report(y, rf_predict)) # definitely overfit

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      4838
           1       1.00      0.52      0.68      1450

    accuracy                           0.89      6288
   macro avg       0.93      0.76      0.81      6288
weighted avg       0.90      0.89      0.87      6288



### Import Data with More Principal Components

In [2]:
df = pd.read_csv(r'C:\Users\sleep\OneDrive\Desktop\Guidehouse\adv_sol\gsa_aiml\features2.csv')

In [3]:
print(len(df))
df.head()

7879


,Unnamed: 0,id,clause,class,clause_clean,clause_tv,clause_tv_stemmed,pc1,pc2,pc3,...,pc791,pc792,pc793,pc794,pc795,pc796,pc797,pc798,pc799,pc800
0,0,1588,18. Governing Law: This Agreement shall be gov...,0,governing law this agreement shall be govern...,"['governing', 'law', 'agreement', 'shall', 'ac...","['govern', 'law', 'agreement', 'shall', 'accor...",-0.036247,0.169982,0.271193,...,0.000182,0.005195,0.021198,-0.002412,-0.012101,-0.006247,-0.001323,-0.003125,0.026405,0.018664
1,1,1146,"1.8 Modification. We may modify, update, or di...",1,modification we may modify update or disco...,"['modification', 'may', 'modify', 'update', 'd...","['modif', 'may', 'modifi', 'updat', 'discontin...",0.030888,0.010890,-0.085120,...,0.031570,0.021787,0.014874,-0.037015,0.029649,0.008136,0.022595,0.016808,0.050276,0.005419
2,2,4792,Except as otherwise expressly provided in this...,0,except as otherwise expressly provided in this...,"['except', 'otherwise', 'expressly', 'provided...","['except', 'otherwis', 'expressli', 'provid', ...",-0.053776,0.139832,0.158595,...,-0.007488,-0.000812,0.008460,-0.012286,-0.001851,0.004413,-0.002884,-0.006213,0.000221,-0.002678
3,3,2759,8.3. The benefit and burdens of this Ag...,1,the benefit and burdens of this agreement ...,"['benefit', 'agreement', 'may', 'assigned', 't...","['benefit', 'agreement', 'may', 'assign', 'tra...",-0.037347,0.069780,0.054379,...,0.026184,-0.007776,0.015586,-0.009922,-0.005702,-0.001346,-0.011155,-0.011686,0.031975,0.010669
4,4,4400,DEFINITIONS,0,definitions,[],[],0.089683,0.032011,-0.018232,...,0.000454,-0.000066,-0.000638,-0.000071,0.001186,0.000016,0.000399,0.000328,0.000847,-0.000425


In [3]:
pca = df.iloc[:, 7:-1]
y = df['class']

### Re-running Logistic Regression

In [7]:
lr = LogisticRegression(solver = 'saga', random_state = 123)
C = uniform(loc=0, scale=10)
hyperparams = dict(C = C, penalty = ['l1', 'l2'])

In [8]:
mod1 = RandomizedSearchCV(lr, hyperparams, random_state = 123, n_iter = 460, scoring = 'f1', cv = 5)

In [9]:
mod1_best = mod1.fit(pca, y)

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

...

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not c

c:\users\sleep\anaconda3\lib\site-packages\sklearn\linear_model\sag.py:337: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [10]:
print('Best params:', mod1_best.best_estimator_.get_params())

Best params: {'C': 9.249066855777176, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 100, 'multi_class': 'warn', 'n_jobs': None, 'penalty': 'l1', 'random_state': 123, 'solver': 'saga', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}


In [11]:
print('Best f1:', mod1_best.best_score_)

Best f1: 0.48714720490713714


In [8]:
mod1_best = LogisticRegression(penalty = 'l1', C = 9.249066855777176, solver = 'saga', random_state = 123, )
lr_mod = mod1_best.fit(pca, y)
lr_predict = lr_mod.predict(pca)

In [9]:
print(classification_report(y, lr_predict))

              precision    recall  f1-score   support

           0       0.90      0.96      0.93      6407
           1       0.76      0.52      0.62      1472

    accuracy                           0.88      7879
   macro avg       0.83      0.74      0.77      7879
weighted avg       0.87      0.88      0.87      7879



### Re-running Random Forest

In [4]:
rf = RandomForestClassifier(criterion = 'gini', random_state = 123)
rf_params = {'max_depth': [1, 3, 5, 10],
             'n_estimators': [100, 200, 300],
             'min_samples_leaf': [1, 3, 5, 10, 15]}

In [5]:
mod2 = RandomizedSearchCV(rf, rf_params, random_state = 123, n_iter = 460, scoring = 'f1', cv = 5)

In [6]:
mod2_best = mod2.fit(pca, y)

c:\users\sleep\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 60 is smaller than n_iter=460. Running 60 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
c:\users\sleep\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1437: Undefined

In [7]:
print('Best params:', mod2_best.best_estimator_.get_params())

Best params: {'bootstrap': True, 'class_weight': None, 'criterion': 'gini', 'max_depth': 10, 'max_features': 'auto', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 123, 'verbose': 0, 'warm_start': False}


In [10]:
print('Best f1:', mod2_best.best_score_)

Best f1: 0.027741870870503234


In [11]:
mod2_best = RandomForestClassifier(
                                   criterion = 'gini',
                                   max_depth = 10,
                                   max_leaf_nodes = None,
                                   min_samples_leaf = 1,
                                   min_samples_split = 2,
                                   n_estimators = 100,
                                   random_state = 123)
rf_mod = mod2_best.fit(pca, y)
rf_predict = rf_mod.predict(pca)

In [12]:
print(classification_report(y, rf_predict))

              precision    recall  f1-score   support

           0       0.89      1.00      0.94      6407
           1       1.00      0.47      0.64      1472

    accuracy                           0.90      7879
   macro avg       0.94      0.74      0.79      7879
weighted avg       0.91      0.90      0.89      7879

